In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from scipy import optimize
from sklearn.linear_model import LinearRegression

In [ ]:
# weather = pd.read_csv('predictions.csv', parse_dates=['time'], index_col='time')

# # Load the CSV file into a pandas DataFrame
# # 'timestamp' column is parsed as dates and set as the index
# df = pd.read_csv('cop_january.csv', parse_dates=['timestamp'], index_col='timestamp')

# # Ensure the index is in datetime format
# if not isinstance(df.index, pd.DatetimeIndex):
#     df.index = pd.to_datetime(df.index)

# # Calculate COP (Coefficient of Performance) for each group
# # COP is the ratio of heat output to electricity input
# df['top20cop'] = df['top20heat'] / df['top20elec']
# df['top50cop'] = df['top50heat'] / df['top50elec']
# df['allcop'] = df['allheat'] / df['allelec']

# # Calculate COP for the differences between all data and top 20%/50%
# df['all_minus_top20_elec'] = df['allelec'] - df['top20elec']
# df['all_minus_top20_heat'] = df['allheat'] - df['top20heat']
# df['all_minus_top20_cop'] = df['all_minus_top20_heat'] / df['all_minus_top20_elec']

# df['all_minus_top50_elec'] = df['allelec'] - df['top50elec']
# df['all_minus_top50_heat'] = df['allheat'] - df['top50heat']
# df['all_minus_top50_cop'] = df['all_minus_top50_heat'] / df['all_minus_top50_elec']

# # Ensure index is in datetime format (redundant if done earlier)
# df.index = pd.to_datetime(df.index)
# weather.index = pd.to_datetime(weather.index)

# # Handle timezone information to ensure consistency between datasets
# if df.index.tz is not None and weather.index.tz is None:
#     weather.index = weather.index.tz_localize(df.index.tz)
# elif df.index.tz is None and weather.index.tz is not None:
#     df.index = df.index.tz_localize(weather.index.tz)
# elif df.index.tz != weather.index.tz:
#     weather.index = weather.index.tz_convert(df.index.tz)

# # Get the date range from the energy consumption dataframe
# start_date = df.index.min()
# end_date = df.index.max()

# # Clip the weather dataframe to match the energy consumption date range
# weather = weather.loc[start_date:end_date]

# # Merge the energy consumption and weather dataframes
# merged_df = pd.merge(df, weather, left_index=True, right_index=True, how='left')

# # Drop the last row in the merged dataframe
# merged_df = merged_df.drop(merged_df.index[-1])
# merged_df.to_csv('COP_&_temperature_for_jamuary_2024.csv')

In [ ]:
merged_df = pd.read_csv('COP_&_temperature_for_jamuary_2024.csv', parse_dates=['timestamp'], index_col='timestamp')

In [ ]:
# Define Richards function for curve fitting
def richards(x, A, K, B, M, v):
    return A + (K - A) / (1 + v * np.exp(-B * (x - M))) ** (1 / v)

# Function to calculate R-squared
def calculate_r_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

# Function to fit Richards curve to data
def fit_richards(x, y):
    # Initialize parameters
    A_init = min(y)
    K_init = max(y)
    B_init = 0.1
    M_init = np.median(x)
    v_init = 5.0
    p0 = [A_init, K_init, B_init, M_init, v_init]
   

    # Set bounds for parameters
    bounds = ([min(y), max(y), 0, min(x), 0.1],
              [max(y), max(y)*1.5, 10, max(x), 10])
    
    
   
    try:
        # Attempt to fit the curve
        popt, _ = optimize.curve_fit(richards, x, y, p0, bounds=bounds, method='trf', maxfev=5000)
        # Calculate R-squared
        y_pred = richards(x, *popt)
        r_squared = calculate_r_squared(y, y_pred)
        return popt, r_squared
    except RuntimeError:
        print("Could not fit Richards function. Using initial guess.")
        return p0, None

# Function to fit linear regression
def fit_linear(x, y):
    model = LinearRegression()
    x_array = np.array(x).reshape(-1, 1)
    y_array = np.array(y)
    model.fit(x_array, y_array)
    y_pred = model.predict(x_array)
    r_squared = calculate_r_squared(y_array, y_pred)
    return model.coef_[0], model.intercept_, r_squared

# List of COP columns to plot
cop_columns = [
    'top20cop', 'top50cop', 'allcop',
    'all_minus_top20_cop', 'all_minus_top50_cop'
]

# Create the main figure
fig = go.Figure()

# Color palette for scatter plots and fit lines
colors = ['blue', 'red', 'green', 'purple', 'orange']

# Dictionary to store Richards parameters and R-squared values
richards_params = {}
r_squared_values = {}

# Dictionary to store linear fit parameters and R-squared values
linear_params = {}
linear_r_squared_values = {}

# Create scatter plots with Richards and linear fit lines
for i, col in enumerate(cop_columns):
    x = merged_df['apparent_temperature (°C)']
    y = merged_df[col]
   
    # Add scatter plot
    fig.add_trace(
        go.Scatter(
            x=x, y=y,
            mode='markers',
            name=f'{col} Data',
            marker=dict(color=colors[i], size=5, opacity=0.6),
        )
    )
   
    # Prepare x values for fit lines
    x_fit = np.linspace(-20, 30, 500)
   
    # Fit Richards curve and add to plot
    try:
        popt_richards, r_squared = fit_richards(x, y)
        y_fit_richards = richards(x_fit, *popt_richards)
        fig.add_trace(
            go.Scatter(x=x_fit, y=y_fit_richards, mode='lines', name=f'{col} Richards Fit', line=dict(color=colors[i], dash='solid'))
        )
       
        # Store parameters and R-squared
        key = col.replace('cop', '').strip('_')
        richards_params[key] = tuple(popt_richards)
        r_squared_values[key] = r_squared
       
    except Exception as e:
        print(f"Could not fit Richards for {col}: {str(e)}")
    
    # Fit linear regression and add to plot
    slope, intercept, linear_r_squared = fit_linear(x, y)
    y_fit_linear = slope * x_fit + intercept
    fig.add_trace(
        go.Scatter(x=x_fit, y=y_fit_linear, mode='lines', name=f'{col} Linear Fit', line=dict(color=colors[i], dash='dot'))
    )
    
    # Store linear fit parameters and R-squared
    linear_params[key] = (slope, intercept)
    linear_r_squared_values[key] = linear_r_squared

# Update layout
fig.update_layout(
    height=600,
    width=1000,
    title_text="COP vs apparent_temperature (°C) (with Richards and Linear Fit Lines)",
    xaxis_title="apparent_temperature (°C)",
    yaxis_title="COP",
    
    yaxis_range=[0, 7],
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    margin=dict(t=50)
)

# Show the plot
fig.show()

# Calculate and print average COP for each category
print("\nAverage COP for each category:")
for col in cop_columns:
    avg_cop = merged_df[col].mean()
    print(f"{col}: {avg_cop:.4f}")

# Add the additional entry for 'all_minus_top50_minus_1'
richards_params['all_minus_top50_minus_1'] = richards_params['all_minus_top50']
r_squared_values['all_minus_top50_minus_1'] = r_squared_values['all_minus_top50']
linear_params['all_minus_top50_minus_1'] = linear_params['all_minus_top50']
linear_r_squared_values['all_minus_top50_minus_1'] = linear_r_squared_values['all_minus_top50']

# Output Richards function parameters in the specified format
print("\nRichards function parameters:")
print("richards_params = {")
for key, params in richards_params.items():
    print(f"    '{key}': {params},")
print("}")

# Print R-squared values for Richards fits
print("\nR-squared values for Richards fits:")
for key, r_squared in r_squared_values.items():
    if r_squared is not None:
        print(f"{key}: {r_squared:.4f}")
    else:
        print(f"{key}: Could not calculate R-squared")

# Output Linear function parameters
print("\nLinear function parameters (slope, intercept):")
print("linear_params = {")
for key, params in linear_params.items():
    print(f"    '{key}': {params},")
print("}")

# Print R-squared values for Linear fits
print("\nR-squared values for Linear fits:")
for key, r_squared in linear_r_squared_values.items():
    print(f"{key}: {r_squared:.4f}")

# Print some basic statistics about the temperature data
print("\napparent_temperature (°C) statistics:")
print(merged_df['apparent_temperature (°C)'].describe())

In [ ]:
# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from scipy import optimize

# # Define Richards function for curve fitting
# def richards(x, A, K, B, M, v):
#     return A + (K - A) / (1 + v * np.exp(-B * (x - M))) ** (1 / v)

# # Function to calculate R-squared
# def calculate_r_squared(y_true, y_pred):
#     ss_res = np.sum((y_true - y_pred) ** 2)
#     ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
#     r_squared = 1 - (ss_res / ss_tot)
#     return r_squared

# # Function to fit Richards curve to data
# def fit_richards(x, y):
#     A_init, K_init = min(y), max(y)
#     B_init, M_init, v_init = 0.1, np.median(x), 1.0
#     p0 = [A_init, K_init, B_init, M_init, v_init]
#     bounds = ([min(y), max(y), 0, min(x), 0.1], [max(y), max(y)*1.5, 10, max(x), 10])
#     try:
#         popt, _ = optimize.curve_fit(richards, x, y, p0, bounds=bounds, method='trf', maxfev=5000)
#         y_pred = richards(x, *popt)
#         r_squared = calculate_r_squared(y, y_pred)
#         return popt, r_squared
#     except RuntimeError:
#         print("Could not fit Richards function. Using initial guess.")
#         return p0, None

# # Function to calculate gradient
# def calculate_gradient(x, y, params):
#     epsilon = 1e-6
#     y1 = richards(x - epsilon, *params)
#     y2 = richards(x + epsilon, *params)
#     return (y2 - y1) / (2 * epsilon)

# # Manual gradient adjustments
# manual_gradients = {
#     'top20': {'left': 0.05, 'right': None},  # Adjust left gradient for top20
#     'top50': {'left': None, 'right': None},
#     'all': {'left': None, 'right': None},
#     'all_minus_top20': {'left': None, 'right': None},
#     'all_minus_top50': {'left': None, 'right': None},
# }

# # List of COP columns to plot
# cop_columns = [
#     'top20cop', 'top50cop', 'allcop',
#     'all_minus_top20_cop', 'all_minus_top50_cop'
# ]

# # Create the main figure
# fig = go.Figure()

# # Color palette for scatter plots and fit lines
# colors = ['blue', 'red', 'green', 'purple', 'orange']

# # Dictionary to store Richards parameters and R-squared values
# richards_params = {}
# r_squared_values = {}

# # Create scatter plots with Richards fit line and linear extrapolation
# for i, col in enumerate(cop_columns):
#     x = merged_df['apparent_temperature (°C)']
#     y = merged_df[col]
   
#     # Add scatter plot
#     fig.add_trace(
#         go.Scatter(
#             x=x, y=y,
#             mode='markers',
#             name=f'{col} Data',
#             marker=dict(color=colors[i], size=5, opacity=0.6),
#         )
#     )
   
#     # Fit Richards curve
#     try:
#         popt_richards, r_squared = fit_richards(x, y)
        
#         # Sort x and y
#         sorted_indices = np.argsort(x)
#         x_sorted = x[sorted_indices]
#         y_sorted = y[sorted_indices]
        
#         # Prepare x values for fit lines
#         x_fit_left = np.linspace(-20, x_sorted[0], 50)
#         x_fit_middle = np.linspace(x_sorted[0], x_sorted[-1], 100)
#         x_fit_right = np.linspace(x_sorted[-1], 20, 50)
        
#         # Calculate gradients at points one away from the edges
#         grad_left = calculate_gradient(x_sorted[1], y_sorted[1], popt_richards)
#         grad_right = calculate_gradient(x_sorted[-2], y_sorted[-2], popt_richards)
        
#         # Apply manual gradient adjustments if specified
#         key = col.replace('cop', '').strip('_')
#         if manual_gradients[key]['left'] is not None:
#             grad_left = manual_gradients[key]['left']
#         if manual_gradients[key]['right'] is not None:
#             grad_right = manual_gradients[key]['right']
        
#         # Calculate y values
#         y_fit_left = richards(x_sorted[0], *popt_richards) + grad_left * (x_fit_left - x_sorted[0])
#         y_fit_middle = richards(x_fit_middle, *popt_richards)
#         y_fit_right = richards(x_sorted[-1], *popt_richards) + grad_right * (x_fit_right - x_sorted[-1])
        
#         # Combine x and y values
#         x_fit = np.concatenate((x_fit_left, x_fit_middle, x_fit_right))
#         y_fit = np.concatenate((y_fit_left, y_fit_middle, y_fit_right))
        
#         # Add fit line to plot
#         fig.add_trace(
#             go.Scatter(x=x_fit, y=y_fit, mode='lines', name=f'{col} Fit', line=dict(color=colors[i]))
#         )
       
#         # Store parameters and R-squared
#         richards_params[key] = tuple(popt_richards)
#         r_squared_values[key] = r_squared
       
#     except Exception as e:
#         print(f"Could not fit Richards for {col}: {str(e)}")

# # Update layout
# fig.update_layout(
#     height=600,
#     width=1000,
#     title_text="COP vs apparent_temperature (°C) (with Richards Fit and Adjusted Linear Extrapolation)",
#     xaxis_title="apparent_temperature (°C)",
#     yaxis_title="COP",
#     yaxis_range=[0, 7],
#     xaxis_range=[-20, 20],
#     legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
#     margin=dict(t=50)
# )

# # Show the plot
# fig.show()

# # Calculate and print average COP for each category
# print("\nAverage COP for each category:")
# for col in cop_columns:
#     avg_cop = merged_df[col].mean()
#     print(f"{col}: {avg_cop:.4f}")

# # Add the additional entry for 'all_minus_top50_minus_1'
# richards_params['all_minus_top50_minus_1'] = richards_params['all_minus_top50']
# r_squared_values['all_minus_top50_minus_1'] = r_squared_values['all_minus_top50']

# # Output Richards function parameters in the specified format
# print("\nRichards function parameters:")
# print("richards_params = {")
# for key, params in richards_params.items():
#     print(f"    '{key}': {params},")
# print("}")

# # Print R-squared values
# print("\nR-squared values for Richards fits:")
# for key, r_squared in r_squared_values.items():
#     if r_squared is not None:
#         print(f"{key}: {r_squared:.4f}")
#     else:
#         print(f"{key}: Could not calculate R-squared")

# # Print some basic statistics about the temperature data
# print("\napparent_temperature (°C) statistics:")
# print(merged_df['apparent_temperature (°C)'].describe())

In [ ]:
class HeatPumpDashboard:
    def __init__(self, csv_file):
        self.scenarios = ['top20', 'top50', 'all', 'all_minus_top20', 'all_minus_top50', 'all_minus_top50_minus_1']
        self.scenario_abbr = {'top20': 'top20', 'top50': 'top50', 'all': 'all', 
                              'all_minus_top20': 'A-T20', 'all_minus_top50': 'A-T50', 
                              'all_minus_top50_minus_1': 'A-T50-1'}
        self.colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        self.load_data(csv_file)
        self.create_figures()
        self.create_outputs()

    def richards(self, x, A, K, B, M, v):
        return A + (K - A) / (1 + v * np.exp(-B * (x - M))) ** (1 / v)

    def load_data(self, csv_file):
        try:
            self.df = pd.read_csv(csv_file, parse_dates=['time'], index_col='time')
            
            required_columns = ['predicted_consumption_Hybrid', 'apparent_temperature (°C)']
            for col in required_columns:
                if col not in self.df.columns:
                    raise KeyError(f"Required column '{col}' not found in the CSV file.")
            
            self.df['predicted_consumption_Hybrid'] /= 1000  # Convert to kWh

            self.total_years = (self.df.index[-1] - self.df.index[0]).days / 365.25
            self.target_total_heat_output = 12000 * self.total_years * 0.85

            # Richards function parameters (5-parameter version):
            richards_params = {
                'top20': (2.8995409604519984, 5.8717905405430315, 0.13935736501785154, 1.9323493448484659, 0.10000000000000105),
                'top50': (2.8605674073654623, 5.507528926930081, 0.17115380323533036, 3.170887311885561, 0.10000000010704134),
                'all': (2.5271495624645697, 4.856725576840968, 0.17125877813614995, 2.9853800928071235, 0.10000000002655143),
                'all_minus_top20': (2.4885026545081863, 4.6841470419299265, 0.17778500554053603, 3.1082025534906936, 0.10000000000004104),
                'all_minus_top50': (2.290348085648673, 4.464281297662724, 0.16295400489423612, 2.6922992910420747, 0.10000000000004214),
                'all_minus_top50_minus_1': (2.290348085648673, 4.464281297662724, 0.16295400489423612, 2.6922992910420747, 0.10000000000004214),
            }

            # Calculate 'all' scenario COP first
            A, K, B, M, v = richards_params['all']
            self.df['all_cop'] = self.richards(self.df['apparent_temperature (°C)'], A, K, B, M, v)

            # Calculate baseline heat output using 'all' scenario
            self.df['baseline_heat_output'] = self.df['predicted_consumption_Hybrid'] * self.df['all_cop']

            # Adjust baseline heat output to match target
            adjustment_factor = self.target_total_heat_output / self.df['baseline_heat_output'].sum()
            self.df['baseline_heat_output'] *= adjustment_factor

            for scenario in self.scenarios:
                A, K, B, M, v = richards_params[scenario]
                self.df[f'{scenario}cop'] = self.richards(self.df['apparent_temperature (°C)'], A, K, B, M, v)
                if scenario == 'all_minus_top50_minus_1':
                    self.df[f'{scenario}cop'] -= 1  # Subtract 1 for this specific scenario
                
                # Use the same baseline heat output for all scenarios
                self.df[f'{scenario}_heat_output'] = self.df['baseline_heat_output']
                
                # Calculate electricity input based on scenario-specific COP
                self.df[f'{scenario}_electricity_input'] = self.df[f'{scenario}_heat_output'] / self.df[f'{scenario}cop']

            global final_df
            final_df = self.df  # Save the DataFrame to a global variable

        except Exception as e:
            print(f"Error loading or processing data: {str(e)}")
            raise

    def create_figures(self):
        try:
            self.fig = go.Figure()
            
            for i, scenario in enumerate(self.scenarios):
                self.fig.add_trace(
                    go.Scatter(x=self.df.index, y=self.df[f'{scenario}_electricity_input'], 
                               name=f"{scenario} Electricity Input",
                               line=dict(color=self.colors[i], width=1.5))
                )

            self.fig.add_trace(
                go.Scatter(x=self.df.index, y=self.df['top20_heat_output'], name="Heat Output",
                           line=dict(color='black', width=1.5))
            )

            
            
            self.fig.add_trace(
                go.Scatter(x=self.df.index, y=self.df['top20cop'], name="Top 20 COP",
                           line=dict(color='green', width=1.5)))

            self.fig.update_layout(
                height=600,
                width=1200, 
                title_text="Heat Output and Required Electricity Input Comparison",
                xaxis_title="Date",
                yaxis_title="Energy (kWh)",
                plot_bgcolor='white',
                paper_bgcolor='white',
            )

        except Exception as e:
            print(f"Error creating figures: {str(e)}")
            raise

    def create_outputs(self):
        self.stats_output = widgets.Output()

    def calculate_yearly_stats(self):
        try:
            def yearly_stats(group):
                return pd.Series({
                    **{f'{s}_cop_min': group[f'{s}cop'].min() for s in self.scenarios},
                    **{f'{s}_cop_avg': group[f'{s}cop'].mean() for s in self.scenarios},
                    **{f'{s}_cop_max': group[f'{s}cop'].max() for s in self.scenarios},
                    **{f'{s}_electricity_input_min': group[f'{s}_electricity_input'].min() for s in self.scenarios},
                    **{f'{s}_electricity_input_avg': group[f'{s}_electricity_input'].mean() for s in self.scenarios},
                    **{f'{s}_electricity_input_max': group[f'{s}_electricity_input'].max() for s in self.scenarios},
                    **{f'{s}_heat_output_min': group[f'{s}_heat_output'].min() for s in self.scenarios},
                    **{f'{s}_heat_output_avg': group[f'{s}_heat_output'].mean() for s in self.scenarios},
                    **{f'{s}_heat_output_max': group[f'{s}_heat_output'].max() for s in self.scenarios},
                })

            self.yearly_stats = self.df.groupby(self.df.index.year).apply(yearly_stats)
            return self.yearly_stats
        except Exception as e:
            print(f"Error calculating yearly stats: {str(e)}")
            raise

    def save_data(self):
        try:
            self.yearly_stats.to_excel('yearly_stats.xlsx')
            self.yearly_stats.to_csv('yearly_stats.csv')
            print("Data saved as 'yearly_stats.xlsx' and 'yearly_stats.csv'")
            return self.yearly_stats
        except Exception as e:
            print(f"Error saving data: {str(e)}")
            raise

    def run(self):
        try:
            yearly_stats = self.calculate_yearly_stats()
            display(self.fig)
            
            print(f"Total years: {self.total_years:.2f}")
            print(f"Target total heat output: {self.target_total_heat_output:.2f} kWh")
            for scenario in self.scenarios:
                print(f"\nScenario: {scenario}")
                print(f"Total heat output: {self.df[f'{scenario}_heat_output'].sum():.2f} kWh")
                print(f"Average yearly heat output: {self.df[f'{scenario}_heat_output'].sum() / self.total_years:.2f} kWh")
                print(f"Total electricity input: {self.df[f'{scenario}_electricity_input'].sum():.2f} kWh")
                print(f"Average yearly electricity input: {self.df[f'{scenario}_electricity_input'].sum() / self.total_years:.2f} kWh")
                print(f"Average COP: {self.df[f'{scenario}cop'].mean():.2f}")
            
            df_stats = self.save_data()
            print("\nDetailed statistics have been saved to Excel and CSV files.")
            print("A DataFrame with detailed statistics is available for further analysis.")
            return df_stats
        except Exception as e:
            print(f"Error running dashboard: {str(e)}")

# Usage
dashboard = HeatPumpDashboard('predictions.csv')
df_stats = dashboard.run()
print("\nYou can now use 'df_stats' for further analysis in your notebook or script.")

In [ ]:
final_df.to_csv('heat_normalised_electricity_load_heat_pump.csv')

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Assuming df_stats is your DataFrame with 'time' already as the index

# Define the groups
heat_groups = [
    'top20_heat_output_max', 'top50_heat_output_max', 'all_heat_output_max',
    'all_minus_top20_heat_output_max', 'all_minus_top50_heat_output_max',
    'all_minus_top50_minus_1_heat_output_max'
]

electricity_groups = [
    'top20_electricity_input_max', 'top50_electricity_input_max', 'all_electricity_input_max',
    'all_minus_top20_electricity_input_max', 'all_minus_top50_electricity_input_max',
    'all_minus_top50_minus_1_electricity_input_max'
]

# Create heat output table
heat_table = df_stats[heat_groups]

# Create electricity input table
electricity_table = df_stats[electricity_groups]

# Display the tables with improved formatting
print("Heat Output Table:")
display(HTML(heat_table.to_html(classes='table table-striped table-hover')))

print("\nElectricity Input Table:")
display(HTML(electricity_table.to_html(classes='table table-striped table-hover')))

# If you want to see the raw data, you can still use print()
# print(heat_table)
# print(electricity_table)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, VBox, HTML
import numpy as np

# Assuming final_df is already loaded and converted to integers (watts)
# Define the columns to work with
columns = ['top20_electricity_input', 'top50_electricity_input', 'all_electricity_input',
           'all_minus_top20_electricity_input', 'all_minus_top50_electricity_input',
           'all_minus_top50_minus_1_electricity_input']

# Pre-compute the column names without '_electricity_input'
column_names = [col.replace('_electricity_input', '') for col in columns]

# Pre-compute the base values as a numpy array
base_values = final_df[columns].to_numpy()

# Create a FigureWidget instead of a Figure
fig = go.FigureWidget()

# Add traces to the figure
for i, column in enumerate(columns):
    fig.add_scatter(x=final_df.index, y=base_values[:, i], name=column_names[i], mode='lines')

# Initial plot setup
fig.update_layout(
    title='Grid Load Simulation for 1 Heat Pump',
    xaxis_title='Time',
    yaxis_title='Load (GW)',
    legend_title='Categories'
)

# Function to calculate statistics and update tables
def calculate_statistics(num_heatpumps=1):
    # Scale base values by the number of heat pumps
    updated_values = base_values * num_heatpumps
    
    # Convert updated values back to DataFrame for easier manipulation
    updated_df = pd.DataFrame(updated_values, columns=columns, index=final_df.index)
    
    # Calculate yearly statistics
    yearly_stats = updated_df.resample('Y').agg(['max', 'sum'])
    
    # Calculate cumulative sum for each year and convert from Wh to GWh
    cumsum_gwh = updated_df.resample('Y').apply(lambda x: x.cumsum().iloc[-1]) 
    
    # Add cumulative sum results to the yearly_stats DataFrame
    for col in columns:
        yearly_stats[(col, 'cumsum')] = cumsum_gwh[col]

    # Prepare HTML tables
    max_table_html = "<table><tr><th>Year</th>"
    cum_table_html = "<table><tr><th>Year</th>"
    
    for col in column_names:
        max_table_html += f"<th>{col} Max (GW)</th>"
        cum_table_html += f"<th>{col} Cum. (GWh)</th>"
    
    max_table_html += "</tr>"
    cum_table_html += "</tr>"
    
    for year, row in yearly_stats.iterrows():
        max_table_html += f"<tr><td>{year.year}</td>"
        cum_table_html += f"<tr><td>{year.year}</td>"
        
        for col in column_names:
            full_col = f"{col}_electricity_input"
            max_value = row[(full_col, 'max')]   # Convert to GW
            cum_value = row[(full_col, 'cumsum')]
            
            max_table_html += f"<td>{max_value:.2f}</td>"
            cum_table_html += f"<td>{cum_value:.2f}</td>"
        
        max_table_html += "</tr>"
        cum_table_html += "</tr>"
    
    max_table_html += "</table>"
    cum_table_html += "</table>"
    
    return max_table_html, cum_table_html

# HTML widgets for displaying the tables
max_table_widget = HTML()
cum_table_widget = HTML()

# Function to update both the plot and the statistics tables
def update_visualization(num_heatpumps=1):
    # Update plot
    updated_values = base_values * num_heatpumps
    for i, trace in enumerate(fig.data):
        trace.y = updated_values[:, i]
    
    fig.update_layout(
        title=f'Grid Load Simulation for {num_heatpumps} Heat Pumps'
    )
    
    # Update tables
    max_table_html, cum_table_html = calculate_statistics(num_heatpumps)
    max_table_widget.value = max_table_html
    cum_table_widget.value = cum_table_html

# Create slider with continuous_update set to False
slider = IntSlider(value=1, min=0, max=15, step=1, description='Number of Heat Pumps, millions', continuous_update=False)

# Create the interactive widget and link it to the update function
interact(update_visualization, num_heatpumps=slider)

# Display the plot and tables
VBox([fig, max_table_widget, cum_table_widget])